# Multi-Agent Conversational System: Customer Query Handling & Real-Time Inventory Check via Vector DB

## 1. Description

This project implements an advanced multi-agent conversational system designed to enhance customer service for **"Ashis Home Goods."** The system leverages **Large Language Models (LLMs)** and a **Vector Database** to provide intelligent and efficient responses to customer inquiries.

- **Core LLM Agent** (Azure GPT-4o-mini): Handles natural language queries and responds with product information, comparisons, and general help.
- **Inventory Agent** (implicit): Performs real-time stock and inventory checks using a **FAISS Vector Database** with embeddings containing product metadata.

**Tech Stack**: Python, Langchain, Azure OpenAI, FAISS, Google Colab Widgets.

## 2. System Architecture

The diagram below illustrates the architecture and data flow:

```mermaid
+---------------------------+
|     👤 USER (Customer)    |
|  (via Google Colab UI)   |
+-----------+--------------+
            |
            v
+---------------------------+
|   🧩 Frontend (Widgets)    |
|                           |
| - Input Box               |
| - Submit Button           |
| - Output Display          |
| - Feedback Buttons        |
| - Download Log Button     |
+-----------+--------------+
            |
            v
+---------------------------+
| 🔁 handle_query_submission |
|     (Python Backend)      |
+-----------+--------------+
            |
     +------|------+
     |             |
     v             v
+-------------+   +--------------------+
| 🧠 Meta-Query? |   | 🤖 Not Meta-Query  |
+-------------+   +--------------------+
     |                  |
     v                  v
+----------------+   +----------------------+
|  Session Memory |   |   Langchain RAG     |
|  (History-based)|   |     Chain            |
+--------+-------+   +----------+-----------+
         |                      |
         v                      v
+----------------+   +----------------------+
| Answer from     |   |  🔍 Vector DB        |
| memory context  |   |  Retriever (FAISS)  |
+----------------+   +----------+-----------+
                               |
                               v
                     +------------------------+
                     |  🧾 Vector Store (FAISS) |
                     |  - Product Embeddings  |
                     |  - Inventory Embeddings|
                     +------------------------+
                               |
                               v
                     +------------------------+
                     | 🤖 LLM Agent (GPT-4o-mini)|
                     | Generates final response|
                     +------------------------+
                               |
                               v
            +---------------------------------+
            |  ✅ Final Answer to UI Output    |
            +---------------------------------+
                               |
         +---------------------+---------------------+
         |                                           |
         v                                           v
+------------------------+         +------------------------------+
| Feedback Logging (CSV) |         | Query + Response Logging     |
|  ("Helpful" / "Not")   |         | (with context) to query_log  |
+------------------------+         +------------------------------+


```

### Flow Explanation

1. **User Interaction**: User types and submits a query.
2. **Query Handler**: `handle_query_submission()` receives and processes the query.
3. **Meta-Query Check**:
    - If it's a meta-query (e.g., "what was my last question?"), it's routed to `Session Memory Logic`.
    - Memory logic fetches and formats relevant history.
4. **RAG Processing**:
    - If not a meta-query, the query enters the **Langchain RAG Chain**.
    - The **Retriever** searches the **FAISS Vector Store** for relevant context (stock, product info).
    - Context + Query are passed to the **LLM Agent** (GPT-4o-mini).
    - The LLM generates a grounded and context-aware answer.
5. **Response**: Displayed to the user in the UI.
6. **Logging**: All interactions are saved to logs for future improvements.
7. **Data Backend**: Product and inventory CSVs → Merged → Chunked → Embedded → Stored in FAISS.

---

## 3. Features

- ✅ **Natural Language Understanding**  
  Handles varied user queries using GPT-4o-mini.

- 📦 **Real-Time Inventory Checks**  
  Provides stock status, quantity, warehouse info, and restock dates.

- 📄 **Contextual Product Information**  
  Shows product descriptions, pricing, features, and warranty.

- 🔍 **Product Comparison**  
  Compares multiple products on features, price, availability (if context allows).

- 🧠 **Session Memory**
  - Remembers queries: "What was my last question?"
  - Recalls product searches: "What did I search for earlier?"

- 💬 **Interactive UI (Google Colab)**
  - Query input box
  - Output display
  - Helpful/Not Helpful buttons
  - Quick action suggestions

- 📊 **Query Logging**
  - Saves queries, responses, retrieved context to `query_log.csv`.
  - Can be downloaded via a button.

- 👍 **Feedback Collection**
  - Stores user ratings in `feedback_log.csv`.

- ❓ **Suggested Follow-ups**
  - Agent recommends next possible questions.

- 🚧 **Graceful Fallbacks**
  - If info isn't found, the agent responds politely with alternatives.

- 📝 **Markdown Output**
  - Structured, readable answers with lists, bolding, and formatting.

---

> This system enables seamless, intelligent product support using cutting-edge retrieval + generation techniques, ensuring that customer queries are answered accurately with up-to-date product data.


In [1]:
!pip install langchain langchain_openai langchain_community faiss-cpu pandas openai tiktoken ipywidgets azure-identity

In [6]:
import os
import pandas as pd
import numpy as np
import io
import time # Globally imported
import datetime
import csv

from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import files
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from functools import partial

# =================== Configuration Settings ===================
# User-provided Azure configurations
# !!! IMPORTANT: ENSURE THESE ARE YOUR ACTUAL VALID CREDENTIALS !!!
AZURE_API_KEY = ""
AZURE_ENDPOINT = ""
CHAT_DEPLOYMENT_NAME = ""
CHAT_MODEL_NAME = ""
CHAT_API_VERSION = ""
EMBEDDING_DEPLOYMENT_NAME = ""
EMBEDDING_MODEL_NAME = ""
EMBEDDING_API_VERSION = ""

# Global variables
products_df = None
inventory_df = None
merged_df = None
embeddings = None
vectorstore = None
retriever = None
llm = None
prompt = None
rag_chain = None
query_history = []
download_in_progress = False

# For logging
QUERY_LOG_FILE = 'query_log.csv'
FEEDBACK_LOG_FILE = 'feedback_log.csv'

# To store current interaction context
current_interaction_context = {}
log_download_output = widgets.Output()
feedback_status_output = widgets.Output()

# =================== Create Vast Product Data ===================
def load_internal_product_data():
    global products_df, inventory_df
    products_data_csv = """product_id,name,category,price,description,features,warranty_info,related_product_ids
1,AlphaPro Noise-Cancelling Headphones,Electronics,199.99,"Studio-quality sound with advanced active noise cancellation. Over-ear design for maximum comfort.","ANC, Bluetooth 5.2, 40-hour battery, USB-C charging, Built-in mic, Foldable design","2-year limited warranty","2,15"
2,AlphaPro Compact Bluetooth Speaker,Electronics,79.99,"Portable and powerful sound with deep bass. IPX7 waterproof.","Bluetooth 5.0, 15-hour battery, IPX7, Stereo pairing, Integrated loop","1-year limited warranty","1,9"
3,OmegaView 55" 4K QLED Smart TV,Electronics,749.99,"Stunning 4K QLED display with vibrant colors and Quantum Dot technology. Integrated streaming services.","4K QLED, HDR10+, Smart Hub, Voice Remote, 3 HDMI, 2 USB, Slim design","1-year manufacturer warranty","4,22"
4,OmegaView Pro Soundbar 5.1,Electronics,299.99,"Immersive 5.1 channel surround sound with wireless subwoofer. Dolby Atmos support.","5.1ch, Wireless Sub, Dolby Atmos, Bluetooth, HDMI eARC, Optical input","1-year manufacturer warranty","3"
5,ProGamer Mechanical Keyboard X1,Gaming,129.99,"RGB backlit mechanical keyboard with customizable switches and macros. Full N-key rollover.","Cherry MX Switches (selectable), RGB per-key, Aluminum frame, Detachable USB-C cable, Macro recording","2-year warranty","6,7"
6,ProGamer High-DPI Mouse M1,Gaming,69.99,"Ultra-lightweight gaming mouse with 16000 DPI optical sensor and programmable buttons.","16000 DPI, 6 programmable buttons, RGB lighting, Lightweight design, On-board memory","2-year warranty","5,7"
7,ProGamer XL Gaming Mousepad,Gaming,29.99,"Extended mousepad with anti-fray stitching and smooth surface for precise tracking.","900x400mm, Optimized surface, Non-slip rubber base, Water-resistant","6-month warranty","5,6"
8,ErgoComfort Office Chair Pro,Furniture,349.99,"Ergonomic mesh office chair with adjustable lumbar support, headrest, and 4D armrests.","Breathable mesh, Adjustable lumbar, Headrest, 4D armrests, Tilt lock, Heavy-duty base","5-year frame, 2-year parts","29"
9,PureFlow HEPA Air Purifier L,Appliances,179.99,"True HEPA filter captures 99.97% of airborne particles. Ideal for large rooms (500 sq ft).","True HEPA, Activated Carbon, Covers 500 sq ft, Auto mode, Air quality sensor, Quiet operation","1-year warranty","10"
10,PureFlow HEPA Filter Replacement,Appliances,39.99,"Official replacement filter for PureFlow HEPA Air Purifier L. Recommended change every 6-8 months.","HEPA H13 grade, Easy to install","N/A","9"
11,RoboVac SmartClean X500,Appliances,279.99,"Robot vacuum with smart navigation, app control, and scheduling. Works with Alexa/Google Assistant.","Smart navigation, App control, 120 min runtime, Self-charging, Anti-drop sensors, Multi-surface cleaning","1-year warranty","30"
12,ChefMaster Pro Blender V12,Kitchen,149.99,"High-speed blender with 1200W motor, 6 stainless steel blades, and multiple preset programs.","1200W, 1.8L Tritan Jar, 6 blades, 5 presets (smoothie, ice crush), Pulse function, Tamper included","3-year motor, 1-year parts","13"
13,ChefMaster Glass Jar (1.8L) for V12,Kitchen,29.99,"Replacement or additional glass jar for ChefMaster Pro Blender V12.","1.8L Borosilicate Glass, Lid and cap included","N/A","12"
14,CozyNight Queen Memory Foam Mattress,Bedroom,499.99,"10-inch queen mattress with gel-infused memory foam for cooling comfort and pressure relief.","Gel memory foam, Medium-firm, Hypoallergenic cover, CertiPUR-US certified, 10-year warranty","10-year limited warranty","15"
15,CozyNight Waterproof Mattress Protector (Queen),Bedroom,34.99,"Protects your mattress from spills, stains, and allergens. Soft and breathable.","Waterproof, Hypoallergenic, Fits up to 16-inch depth, Machine washable","5-year warranty","1,14"
16,AdventureSeeker 4-Person Dome Tent,Outdoor,129.99,"Easy setup dome tent for 4 people. Weatherproof with rainfly included.","Sleeps 4, WeatherTec system, Easy setup poles, Carry bag included","1-year warranty","17"
17,AdventureSeeker Camping Sleeping Bag (3-Season),Outdoor,49.99,"Comfortable and warm 3-season sleeping bag. Rated for 30°F to 50°F.","Polyester fill, Comfort cuff, ZipPlow zipper, Machine washable","1-year warranty","16"
18,ReadWell PaperLight E-Reader Pro,Electronics,139.99,"6.8-inch glare-free e-ink display with adjustable warm light. Waterproof (IPX8).","6.8” E-ink Carta, 300 ppi, Adjustable warm light, IPX8 Waterproof, 8GB storage, Wi-Fi","1-year warranty","19,20"
19,ClassicLeather E-Reader Cover (Pro),Accessories,24.99,"Premium leather cover for ReadWell PaperLight E-Reader Pro. Auto sleep/wake.","Genuine leather, Magnetic closure, Auto sleep/wake, Multiple colors","90-day warranty","18"
20,"The Alchemist by Paulo Coelho (eBook)",Books,9.99,"Digital eBook version of the international bestseller.","Fiction, Philosophical, Inspirational","N/A","18"
21,"Sapiens: A Brief History of Humankind by Yuval Noah Harari (eBook)",Books,12.99,"Digital eBook version of the acclaimed non-fiction work.","Non-Fiction, History, Anthropology","N/A","18"
22,StreamCast Ultra 4K Streaming Stick,Electronics,49.99,"Plug-and-play 4K HDR streaming with Dolby Vision and Atmos. Voice remote with TV controls.","4K HDR, Dolby Vision/Atmos, Wi-Fi 6, Voice remote, Access to major streaming apps","1-year warranty","3"
23,FitTrack Pro Smartwatch,Wearables,159.99,"Advanced smartwatch with GPS, heart rate monitoring, SpO2, sleep tracking, and 5ATM water resistance.","AMOLED display, GPS, HR, SpO2, Sleep tracking, 5ATM, 14-day battery life, 100+ sport modes","1-year warranty","24"
24,"FitTrack Pro Replacement Bands (Silicone, 3-Pack)",Accessories,19.99,"Set of 3 silicone replacement bands for FitTrack Pro Smartwatch. Various colors.","Soft silicone, Quick release pins, Breathable","N/A","23"
25,AeroPress Go Portable Coffee Maker,Kitchen,31.95,"Compact and portable coffee maker, brews Americano, espresso, and cold brew style coffee.","Brews 1-3 cups, Microfilter, BPA-free, Mug with lid included, Lightweight","1-year warranty","26"
26,"ArtisanRoast Whole Bean Coffee (Medium Roast, 12oz)",Groceries,14.99,"Premium Arabica whole bean coffee, medium roast with notes of chocolate and caramel.","100% Arabica, Small batch roasted, Ethically sourced","N/A","25"
27,YogaFlow Essential Mat (5mm),Fitness,29.99,"Eco-friendly TPE yoga mat, non-slip surface, 5mm thick for comfort and stability.","TPE material, Non-slip, Lightweight, Carry strap included, 72x24 inches","6-month warranty","28"
28,YogaFlow Cork Yoga Block,Fitness,14.99,"Sustainable cork yoga block for support and stability in various poses.","Natural cork, Sturdy, Beveled edges","N/A","27"
29,FlexiDesk Standing Desk Converter,Furniture,189.99,"Adjustable height standing desk converter, fits on top of existing desks. Dual monitor compatible.","Adjustable height (5-17 inches), Gas spring lift, Keyboard tray, Supports up to 35 lbs","3-year warranty","8"
30,AquaPure Smart Water Bottle,Gadgets,59.99,"Self-cleaning water bottle with UV-C sterilization. Tracks water intake via app.","UV-C sterilization, Stainless steel, 20oz, App integration, Rechargeable battery, Glow reminders","1-year warranty","11"
"""
    inventory_data_csv = """inventory_id,product_id,warehouse,quantity,last_restock_date
1,1,North,110,2025-05-15
2,2,North,75,2025-05-10
3,3,East,30,2025-04-05
4,4,West,0,2025-04-20
5,5,South,90,2025-05-01
6,6,North,150,2025-05-15
7,7,East,200,2025-05-02
8,8,West,45,2025-03-30
9,9,South,60,2025-05-05
10,10,East,120,2025-04-20
11,11,North,0,2025-02-10
12,12,West,85,2025-05-18
13,13,South,40,2025-04-25
14,14,North,70,2025-05-01
15,15,East,100,2025-03-15
16,16,West,25,2025-04-11
17,17,South,65,2025-05-13
18,18,North,95,2025-05-08
19,19,East,150,2025-04-28
20,20,Global,10000,2025-01-01
21,21,Global,10000,2025-01-01
22,22,West,55,2025-05-19
23,23,South,0,2025-03-01
24,24,North,180,2025-05-10
25,25,East,70,2025-04-22
26,26,West,90,2025-05-17
27,27,South,110,2025-05-03
28,28,North,200,2025-04-19
29,29,East,35,2025-03-25
30,30,West,50,2025-05-09
"""
    products_df = pd.read_csv(io.StringIO(products_data_csv))
    inventory_df = pd.read_csv(io.StringIO(inventory_data_csv))
    print(f"Internal product data loaded: {len(products_df)} products, {len(inventory_df)} inventory records.")

# =================== Logging Setup ===================
def setup_log_files():
    if not os.path.exists(QUERY_LOG_FILE):
        with open(QUERY_LOG_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['timestamp', 'query', 'response', 'retrieved_context_preview'])
    if not os.path.exists(FEEDBACK_LOG_FILE):
        with open(FEEDBACK_LOG_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['timestamp', 'query', 'response', 'feedback'])

def log_query(query, response, retrieved_context):
    context_preview = "N/A"
    if retrieved_context:
        if isinstance(retrieved_context, list) and retrieved_context and all(hasattr(doc, 'page_content') for doc in retrieved_context):
            context_preview = " | ".join([doc.page_content[:100] + "..." for doc in retrieved_context])
        elif isinstance(retrieved_context, str):
            context_preview = retrieved_context
    try:
        with open(QUERY_LOG_FILE, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([datetime.datetime.now().isoformat(), query, response, context_preview])
    except Exception as e:
        if 'output_area' in globals() and globals().get('output_area'):
            with output_area: display(Markdown(f"<i style='color:orange;'>_Error logging query: {e}_</i>"))
        else:
            print(f"Error logging query (output_area not available): {e}")

def log_feedback_entry(query, response, feedback_value):
    try:
        with open(FEEDBACK_LOG_FILE, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([datetime.datetime.now().isoformat(), query, response, feedback_value])
        with feedback_status_output:
            clear_output(wait=True)
            display(Markdown(f"<i>Feedback '{feedback_value}' logged. Thank you!</i>"))
    except Exception as e:
        with feedback_status_output:
            clear_output(wait=True)
            display(Markdown(f"<i style='color:red;'>Error logging feedback: {e}</i>"))

# =================== Initialize LLM and Embeddings ===================
def initialize_azure_components():
    global embeddings, llm
    if not AZURE_API_KEY or "YOUR_AZURE_OPENAI_API_KEY" in AZURE_API_KEY.upper():
        raise ValueError("Azure API Key seems to be a placeholder or is missing. Please update with your actual key in the Configuration Settings section.")
    if not AZURE_ENDPOINT or "YOUR_AZURE_OPENAI_ENDPOINT" in AZURE_ENDPOINT.upper():
        raise ValueError("Azure Endpoint seems to be a placeholder or is missing. Please update with your actual endpoint in the Configuration Settings section.")

    embeddings = AzureOpenAIEmbeddings(
        azure_deployment=EMBEDDING_DEPLOYMENT_NAME, model=EMBEDDING_MODEL_NAME,
        api_version=EMBEDDING_API_VERSION, api_key=AZURE_API_KEY, azure_endpoint=AZURE_ENDPOINT
    )
    llm = AzureChatOpenAI(
        azure_deployment=CHAT_DEPLOYMENT_NAME, model_name=CHAT_MODEL_NAME,
        api_version=CHAT_API_VERSION, api_key=AZURE_API_KEY, azure_endpoint=AZURE_ENDPOINT,
        temperature=0.1,
    )
    print("Azure components initialized.")

# =================== Process Data and Build Vector Store ===================
def process_data_and_build_vector_store():
    global merged_df, vectorstore, retriever, embeddings
    if products_df is None or inventory_df is None:
        print("Product or inventory data not loaded. Cannot build vector store.")
        return

    products_df['product_id'] = products_df['product_id'].astype(str)
    inventory_df['product_id'] = inventory_df['product_id'].astype(str)

    merged_df = products_df.merge(inventory_df, on='product_id', how='left')

    merged_df['price'] = pd.to_numeric(merged_df['price'], errors='coerce').fillna(0.0)
    merged_df['quantity'] = pd.to_numeric(merged_df['quantity'], errors='coerce').fillna(0).astype(int)

    text_cols_to_fill = ['name', 'category', 'description', 'features', 'warranty_info',
                         'related_product_ids', 'warehouse', 'last_restock_date']
    for col in text_cols_to_fill:
        if col not in merged_df.columns:
            merged_df[col] = 'N/A'
        else:
            merged_df[col] = merged_df[col].fillna('N/A')

    product_texts_local = []
    for _, row in merged_df.iterrows():
        product_text = f"""
Product ID: {row['product_id']}
Name: {row['name']}
Category: {row['category']}
Price: ${row['price']:.2f}
Description: {row['description']}
Features: {row['features']}
Warranty: {row['warranty_info']}
Related Products (IDs): {row['related_product_ids']}
Stock Status: {"In Stock" if row['quantity'] > 0 else "Out of Stock"}
Available Quantity: {row['quantity']}
Warehouse: {row['warehouse']}
Last Restock: {row['last_restock_date']}
        """
        product_texts_local.append(product_text.strip())

    if not product_texts_local: print("No product texts generated."); return
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    docs_for_faiss = text_splitter.create_documents(product_texts_local)

    if not docs_for_faiss: print("No documents for FAISS."); return
    if embeddings is None: print("Embeddings model not initialized."); return

    vectorstore = FAISS.from_documents(docs_for_faiss, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    print(f"Vector store created/rebuilt with {len(docs_for_faiss)} documents.")


# =================== Set up RAG Chain ===================
def setup_rag_chain():
    global prompt, rag_chain, llm, retriever
    if llm is None or retriever is None: print("LLM or Retriever not initialized."); return

    template = """You are an advanced AI customer service agent for 'Ashis Home Goods'.
Your goal is to provide accurate, helpful, and engaging assistance.

**Core Instructions:**
1.  **Use ONLY Provided Context:** Base all answers strictly on the 'Relevant product information' provided. Do not use external knowledge.
2.  **Inventory Specialist Role:** When asked about stock, quantity, or availability, be precise using 'Stock Status', 'Available Quantity', 'Warehouse', and 'Last Restock' from the context.
3.  **Structured Display:** For product details, use Markdown for clarity:
    Example:
    **Product Name:** [Name]
    - **Price:** $[Price]
    - **Status:** [Stock Status] ([Quantity] units available)
    - **Key Features:**
        - Feature 1
        - Feature 2
    - **Warranty:** [Warranty Info]
4.  **Clarification & Disambiguation:** If a query is vague (e.g., "I need a speaker") and the context shows multiple relevant items, ask clarifying questions. (e.g., "We have a few speakers. Are you looking for a portable Bluetooth speaker or a soundbar for your TV?"). Do not list all items unless specifically asked.
5.  **Product Comparison:** If asked to compare products (e.g., "Compare headphones X and speaker Y"), and context for them exists, provide a summary of key differences in features, price, and availability in a structured way.
6.  **Proactive Suggestions:** After answering a query about a product, if appropriate and if context allows (check 'Related Products (IDs)' or infer from features), you MAY suggest ONE related accessory or a highly similar alternative. Phrase it naturally. To do this, you might need to use the product IDs mentioned in 'Related Products (IDs)' to see if their details are also in the current retrieved context.
7.  **Graceful "I Don't Know":** If information isn't in the context, state: "I don't have that specific detail in my current information for [product name/query topic]. Would you like me to check something else about it, or perhaps search for a different product?"
8.  **Suggest Follow-up Questions:** At the end of a substantive response about a product or a comparison, suggest 1-2 relevant follow-up questions the user might have. Prefix each with `SUGGESTED_QUESTION:`. Example:
    `SUGGESTED_QUESTION: What is the battery life of the AlphaPro Headphones?`
    `SUGGESTED_QUESTION: How does the AlphaPro Speaker compare to other portable speakers?`

Relevant product information:
{context}

Customer question: {question}

Your helpful answer:"""
    prompt = ChatPromptTemplate.from_template(template)

    def retrieve_and_format_context(input_dict):
        question = input_dict["question"]
        global current_interaction_context
        if retriever is None: raise ValueError("Retriever is not initialized.")
        retrieved_docs = retriever.invoke(question)
        current_interaction_context['retrieved_docs'] = retrieved_docs
        return {"context": retrieved_docs, "question": question}

    rag_chain = (
        RunnablePassthrough.assign(extracted_inputs=retrieve_and_format_context)
        | (lambda x: {"context": x["extracted_inputs"]["context"], "question": x["extracted_inputs"]["question"]})
        | prompt
        | llm
        | StrOutputParser()
    )
    print("RAG chain set up.")

# =================== UI Elements & Handlers ===================
output_area = widgets.Output()
query_input = widgets.Text(placeholder="Ask about our products...", description="You:", layout=widgets.Layout(width='70%'))
submit_button = widgets.Button(description="Send", button_style='success', icon='paper-plane')
feedback_buttons_box = widgets.HBox([], layout=widgets.Layout(align_items='center', margin='5px 0'))
quick_actions_box = widgets.VBox([], layout=widgets.Layout(align_items='center', width='100%'))

def on_feedback_button_clicked(b, feedback_value):
    log_feedback_entry(current_interaction_context.get('query'), current_interaction_context.get('response'), feedback_value)
    feedback_buttons_box.children = []

def on_quick_action_clicked(b, suggested_query):
    query_input.value = suggested_query
    handle_query_submission(None)

def display_feedback_buttons():
    fb_yes = widgets.Button(description="👍 Helpful", button_style='success', icon='thumbs-up', layout=widgets.Layout(margin='0 5px 0 0'))
    fb_no = widgets.Button(description="👎 Not Helpful", button_style='danger', icon='thumbs-down')
    fb_yes.on_click(partial(on_feedback_button_clicked, feedback_value='helpful'))
    fb_no.on_click(partial(on_feedback_button_clicked, feedback_value='not_helpful'))
    feedback_buttons_box.children = [fb_yes, fb_no]
    feedback_status_output.clear_output()

def parse_and_display_quick_actions(response_text):
    quick_actions = []
    for line in response_text.split('\n'):
        if line.strip().startswith("SUGGESTED_QUESTION:"):
            action_text = line.strip().replace("SUGGESTED_QUESTION:", "").strip()
            if action_text: quick_actions.append(action_text)
    buttons = []
    if quick_actions:
        for action in quick_actions:
            btn = widgets.Button(
                description=action,
                layout=widgets.Layout(
                    width='auto', margin='5px', padding='8px 15px', border='1px solid #b0b0b0'
                ),
                button_style=''
            )
            btn.style.button_color = '#e0f0ff'
            btn.style.font_weight = 'bold'
            btn.on_click(partial(on_quick_action_clicked, suggested_query=action))
            buttons.append(btn)
    quick_actions_box.children = buttons

# MODIFICATION: handle_query_submission with DYNAMIC and ROBUST memory recall
def handle_query_submission(button_instance):
    global current_interaction_context, query_history
    query_original = query_input.value.strip()

    # Sanitize query for keyword matching
    temp_query_lower = query_original.lower()
    chars_to_strip_from_end = "?!., "
    while temp_query_lower and temp_query_lower[-1] in chars_to_strip_from_end:
        temp_query_lower = temp_query_lower[:-1]
    query_for_keyword_matching = temp_query_lower.strip()

    # --- Optional: Uncomment these debug prints if you're still facing issues ---
    # print(f"DEBUG: Original query: '{query_original}'")
    # print(f"DEBUG: Sanitized for matching: '{query_for_keyword_matching}'")
    # print(f"DEBUG: Query history before processing: {query_history}")
    # --- End Debug Prints ---

    with output_area: clear_output(wait=True)
    feedback_buttons_box.children = []
    quick_actions_box.children = []
    feedback_status_output.clear_output()
    log_download_output.clear_output()

    if not query_original:
        with output_area: display(Markdown("<p style='color:orange;'>Please enter a question.</p>")); return
    if query_for_keyword_matching == 'exit':
        with output_area: display(Markdown("👋 Thank you for using Ashis Home Goods support!"))
        query_input.disabled = True; submit_button.disabled = True; return

    response_text = ""
    is_meta_query_handled = False

    # --- Dynamic Meta-Query Detection Logic ---

    # 1. Check for "last question" intent
    if not is_meta_query_handled and \
       ("last question" in query_for_keyword_matching or \
        "last query" in query_for_keyword_matching or \
        query_for_keyword_matching == "my last question" or \
        query_for_keyword_matching == "was my last question") and \
       not ("what product" in query_for_keyword_matching or "which product" in query_for_keyword_matching):
        # print("DEBUG: Detected 'last question' intent")
        is_meta_query_handled = True
        with output_area:
            display(Markdown(f"**You asked:** {query_original}\n\n---"))
            if not query_history:
                response_text = "You haven't asked any questions before this one in this session."
            else:
                response_text = f"Your last question was: \"{query_history[-1]}\""
            display(Markdown(response_text))
            current_interaction_context = {'query': query_original, 'response': response_text, 'retrieved_docs': []}
            log_query(query_original, response_text, "N/A - Meta query for last question")

    # 2. Check for "previous questions" (general list) intent
    if not is_meta_query_handled and \
         ("previous question" in query_for_keyword_matching or # Catches singular "question" and plural "questions"
          "questions i asked" in query_for_keyword_matching or
          "did i ask" in query_for_keyword_matching or
          "have i asked" in query_for_keyword_matching or
          "my questions" in query_for_keyword_matching or # handles "what are my previous questions"
          query_for_keyword_matching == "what are the previous question i asked to you" # specific user example
          ) and \
         "product" not in query_for_keyword_matching: # Differentiate from product history
        # print("DEBUG: Detected 'general previous questions' intent")
        is_meta_query_handled = True
        with output_area:
            display(Markdown(f"**You asked:** {query_original}\n\n---"))
            if not query_history:
                response_text = "You haven't asked any questions before this one in this session."
            else:
                response_text = "**Here are your previous questions (most recent first):**\n"
                num_questions_to_show = min(len(query_history), 5)
                for i in range(num_questions_to_show):
                    response_text += f"- \"{query_history[-(i+1)]}\"\n"
            display(Markdown(response_text))
            current_interaction_context = {'query': query_original, 'response': response_text, 'retrieved_docs': []}
            log_query(query_original, response_text, "N/A - Meta query for general history list")

    # 3. Check for "product search history" intent
    if not is_meta_query_handled and \
         (("product" in query_for_keyword_matching and \
          ("previous question" in query_for_keyword_matching or
           "questions i asked" in query_for_keyword_matching or
           "searched for" in query_for_keyword_matching or
           "asked about" in query_for_keyword_matching or
           "viewed" in query_for_keyword_matching or
           "look for" in query_for_keyword_matching or
           "did i search" in query_for_keyword_matching or
           "checked" in query_for_keyword_matching or # Added "checked"
           "history" in query_for_keyword_matching or
           "earlier" in query_for_keyword_matching or
           "before" in query_for_keyword_matching or
           "asked questions on product" in query_for_keyword_matching)
         ) or query_for_keyword_matching == "what products i checked previously" # Direct match for user's case
        ):
        # print("DEBUG: Detected 'product search history' intent")
        is_meta_query_handled = True
        with output_area:
            display(Markdown(f"**You asked:** {query_original}\n\n---"))
            potential_product_queries = [q for q in query_history]
            if not potential_product_queries:
                response_text = "It doesn't look like you've searched for specific products or asked other types of questions yet in this session."
            else:
                response_text = "**Based on your previous questions (likely product-related), you asked about (most recent first):**\n"
                num_queries_to_show = min(len(potential_product_queries), 5)
                for i in range(num_queries_to_show):
                    response_text += f"- \"{potential_product_queries[-(i+1)]}\"\n"
                if len(potential_product_queries) > num_queries_to_show:
                    response_text += "... and more.\n"
            display(Markdown(response_text))
            current_interaction_context = {'query': query_original, 'response': response_text, 'retrieved_docs': []}
            log_query(query_original, response_text, "N/A - Meta query for product search history")

    if is_meta_query_handled:
        query_input.value = ""
        return # IMPORTANT: Exit after handling a meta-query

    # print(f"DEBUG: No meta-query match. Adding '{query_original}' to history and proceeding to RAG.")
    query_history.append(query_original)

    with output_area:
        display(Markdown(f"**You asked:** {query_original}\n\n---")); display(Markdown("🤖 *Thinking...*"))

    current_interaction_context = {'query': query_original}
    start_time = time.time()

    try:
        if rag_chain is None:
            with output_area: clear_output(wait=True); display(Markdown("<p style='color:red;'>RAG chain not initialized.</p>")); return

        response = rag_chain.invoke({"question": query_original})
        current_interaction_context['response'] = response

        end_time = time.time()
        with output_area:
            clear_output(wait=True)
            display(Markdown(f"**You asked:** {query_original}\n\n---"))
            display(Markdown(f"**Agent's Response (in {end_time - start_time:.2f} seconds):**"))
            display(Markdown(response or "I apologize, I encountered an issue providing a response."))
        log_query(query_original, response, current_interaction_context.get("retrieved_docs", []))
        display_feedback_buttons()
        parse_and_display_quick_actions(response)

    except Exception as e:
        with output_area:
            clear_output(wait=True); display(Markdown(f"**You asked:** {query_original}\n\n---"))
            display(Markdown(f"<p style='color:red;'>🚨 Error: {e}</p>"));
        current_interaction_context['response'] = f"ERROR: {e}"
        log_query(query_original, f"ERROR: {e}", current_interaction_context.get("retrieved_docs", []))
        import traceback
        with output_area: display(Markdown(f"<pre>\n{traceback.format_exc()}\n</pre>"))
    query_input.value = ""


submit_button.on_click(handle_query_submission)
query_input.on_submit(handle_query_submission)

def on_download_query_log_clicked(b):
    global download_in_progress
    if download_in_progress:
        return
    download_in_progress = True
    try:
        with log_download_output:
            clear_output(wait=True)
            if os.path.exists(QUERY_LOG_FILE) and os.path.getsize(QUERY_LOG_FILE) > 0:
                try:
                    display(Markdown(f"<i>Initiating download for '{QUERY_LOG_FILE}'... Check your browser downloads.</i>"))
                    files.download(QUERY_LOG_FILE)
                except Exception as e:
                    display(Markdown(f"<i style='color:red'>Error downloading query log: {e}</i>"))
            else:
                display(Markdown(f"<i>Query log file ('{QUERY_LOG_FILE}') is empty or not found. Make some queries first.</i>"))
    finally:
        time.sleep(0.5)
        download_in_progress = False

def display_main_interface():
    display(Markdown("## 🤖 Ashis Home Goods Support Agent"))
    display(Markdown("Ask about product details, features, price, comparisons, and stock. Type `exit` to quit."))

    input_box = widgets.HBox([query_input, submit_button])
    chat_vbox = widgets.VBox([
        input_box,
        output_area,
        feedback_buttons_box,
        feedback_status_output,
        quick_actions_box
    ])
    logs_title = Markdown("--- \n### 📜 Logs")
    download_query_log_button = widgets.Button(
        description="Download Query Log",
        icon='download',
        button_style='info',
        layout=widgets.Layout(margin='5px 0', width='200px')
    )
    download_query_log_button.on_click(on_download_query_log_clicked)
    logs_display_box = widgets.VBox([download_query_log_button, log_download_output])
    display(chat_vbox)
    display(logs_title)
    display(logs_display_box)

def run_application():
    global query_history, download_in_progress
    query_history = []
    download_in_progress = False
    try:
        initialize_azure_components()
        setup_log_files()
        load_internal_product_data()
        process_data_and_build_vector_store()
        setup_rag_chain()
        display_main_interface()
    except ValueError as e:
        print(f"Configuration Error: {e}")
        if 'output_area' in globals() and globals().get('output_area') is not None:
             with output_area: clear_output(wait=True); display(Markdown(f"<p style='color:red;'><strong>Configuration Error:</strong> {e}</p><p>Please check your Azure credentials and endpoints in the script's Configuration Settings section.</p>"))
        else:
            display(Markdown(f"<p style='color:red;'><strong>Configuration Error:</strong> {e}</p><p>Please check your Azure credentials and endpoints in the script's Configuration Settings section.</p>"))
    except Exception as e:
        print(f"An unexpected error occurred during setup: {e}")
        if 'output_area' in globals() and globals().get('output_area') is not None:
            with output_area: clear_output(wait=True); display(Markdown(f"<p style='color:red;'>An unexpected error occurred during setup: {e}</p>"))
        else:
            display(Markdown(f"<p style='color:red;'>An unexpected error occurred during setup: {e}</p>"))
        import traceback; traceback.print_exc()

# Run the application
run_application()

Azure components initialized.
Internal product data loaded: 30 products, 30 inventory records.
Vector store created/rebuilt with 30 documents.
RAG chain set up.


## 🤖 Ashis Home Goods Support Agent

Ask about product details, features, price, comparisons, and stock. Type `exit` to quit.

--- 
### 📜 Logs